In [1]:
import pandas as pd
import geopandas as geopd

import numpy as np

import os
import pathlib

from tqdm.notebook import tqdm

In [2]:
# TODO add swe when it is done
order_list = ['discharge_vol', 'discharge_spec', 'precipitation', 'pet', 'pet_era5_land', 'pet_fmi', 'snow_evaporation', 'swe', 'swe_cci3-1', 'snow_depth', 'temperature_gmin', 'temperature_min', 'temperature_mean', 'temperature_max', 'humidity_rel', 'radiation_global']

calculate_pet = False

#for bilal
#order_list = ['discharge_vol', 'discharge_spec', 'precipitation', 'snow_depth', 'temperature_gmin', 'temperature_min', 'temperature_mean', 'temperature_max', 'humidity_rel', 'radiation_global']

catchments_path = "/home/iielse/Documents/DIWA/data/CAMELS-FI catchments/CAMELS-FI_catchments.gpkg"
src_root = "/media/iielse/T9/CAMELS-FI/data/timeseries_by_attribute"
dst_root = "/media/iielse/T9/CAMELS-FI/data/timeseries"
catchments = geopd.read_file(catchments_path, layer='v1')
"""
# for bilal
catchments_path = "/home/iiro/Documents/DIWA/bilal/watersheds.gpkg"
src_root = "/home/iiro/Documents/DIWA/bilal/timeseries_by_attribute"
dst_root = "/home/iiro/Documents/DIWA/bilal/timeseries"
catchments = geopd.read_file(catchments_path, layer='watersheds')
"""
root_path = pathlib.Path(src_root)
# Getting the files in 
files = [item for item in root_path.iterdir() if item.is_file()]

### Combining snow evaporation and era5 pet based on if snow depth > 0

In [3]:
def read_daily_timeseries_csv(path, time_column='date'):
    df = pd.read_csv(path)
    df[time_column] = pd.to_datetime(df[time_column], format='mixed') 
    df = df.set_index(time_column)
    return df

# Some of the files contain too many catchments (too short flow time series), so the correct ones are read from the definitely correct place.
gauges = list(catchments['Paikka_Id'])

if calculate_pet:

    for file in files:
        attribute = file.stem

        if attribute == 'pet_fmi':
            pet_fmi = read_daily_timeseries_csv(file)
            pet_fmi = pet_fmi[gauges]
            
        if attribute == 'pet_era5_land':
            pet_era5 = read_daily_timeseries_csv(file)
            pet_era5 = pet_era5[gauges]
        if attribute == 'snow_evaporation':
            snow_e = read_daily_timeseries_csv(file)
            snow_e = snow_e[gauges]
        if attribute == 'snow_depth':
            snow_depth = read_daily_timeseries_csv(file)
            # snow depth has some extra gauges that have been removed
            snow_depth = snow_depth[gauges]
    assert pet_era5.columns.equals(snow_e.columns), "columns don't match"
    assert pet_era5.columns.equals(snow_depth.columns), "columns don't match"
    assert pet_era5.index.equals(snow_e.index), "indices don't match"
    assert pet_era5.index.equals(snow_depth.index), "indices don't match"

    # Snow evaporation is used for snowy days
    pet = pd.DataFrame(np.where(snow_depth > 0, snow_e, np.nan), index=pet_era5.index, columns=pet_era5.columns)
    # Filling non-snowy observations with FMI pet,
    pet = pet.fillna(pet_fmi)
    # then the gaps with era5-land pet
    pet = pet.fillna(pet_era5)

    # Limiting the valid range to 1981, since era5 has so different values from 
    pet = pet.loc["1981":"2023"]
    
    pet_path = os.path.join(src_root, 'pet.csv')
    pet_path = pathlib.Path(pet_path)
    pet.to_csv(pet_path)
    if pet_path not in files:
        files.append(pet_path)
    

In [4]:
for gauge in tqdm(gauges):
    columns = []
    
    for file in files:
        attribute = file.stem
        meteo = read_daily_timeseries_csv(file)

        #column = meteo[[gauge]]
        
        
        column = meteo.loc["1961":"2023", [gauge]]
        column = column.rename({gauge: attribute}, axis=1)
        columns.append(column)
    output = pd.concat(columns, axis=1)
    # reordering the columns
    output = output[order_list]
    dst_path = os.path.join(dst_root, f"CAMELS_FI_hydromet_timeseries_{gauge}_19610101-20231231.csv")
    output.to_csv(dst_path)

  0%|          | 0/320 [00:00<?, ?it/s]

In [17]:
str(meteo.index[0].date())

'1961-01-01'

In [5]:
file

PosixPath('/media/iielse/T9/CAMELS-FI/data/timeseries_by_attribute/.~lock.pet_fmi.csv#')

In [6]:
meteo

,996,1003,1008,1009,1012,1014,1015,1018,1019,1021,...,937,941,943,946,951,961,964,1369,1366,1357
date,,,,,,,,,,,,,,,,,,,,,
1979-01-02,40.46,22.44,10.46,10.00,9.02,2.37,21.17,20.49,11.86,0.00,...,49.88,58.25,57.71,55.48,52.72,30.55,0.0,0.00,0.00,0.00
1979-01-03,39.32,29.04,13.50,12.93,11.69,3.60,22.23,21.70,12.61,0.00,...,50.46,58.68,57.56,55.04,52.52,33.07,0.0,35.16,39.85,30.10
1979-01-04,38.18,35.63,16.54,15.86,14.36,4.84,23.29,22.90,13.36,0.00,...,51.03,59.12,57.41,54.60,52.33,35.59,0.0,70.31,79.71,60.21
1979-01-05,41.66,34.18,15.82,15.17,13.74,4.53,24.82,24.38,19.50,0.78,...,52.22,68.12,65.72,60.30,58.30,38.65,0.0,65.27,75.78,61.91
1979-01-06,45.13,32.73,15.10,14.48,13.11,4.21,26.35,25.85,25.64,1.57,...,53.41,77.12,74.03,66.00,64.28,41.71,0.0,60.23,71.86,63.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-26,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,8.45
2022-05-27,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00
2022-05-28,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00
